# Healthcare Chatbot: Symptom Diagnosis and Precaution Recommendations

Project Made by : HAQ NAWAZ MALIK

# This chatbot is designed to assist users in diagnosing potential health conditions based on their reported symptoms.
# It uses a decision tree classifier and an SVM model trained on a healthcare dataset to make predictions.
# The chatbot also provides descriptions of the predicted conditions and recommended precautions.
# Users can interact with the chatbot through a command-line interface, providing their symptoms and duration.



In [15]:
import re
import pandas as pd
import pyttsx3
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier, _tree
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
import csv
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

training = pd.read_csv('/content/healthcare-chatbot/Data/Training.csv')
testing = pd.read_csv('/content/healthcare-chatbot/Data/Testing.csv')
cols = training.columns
cols = cols[:-1]
x = training[cols]
y = training['prognosis']
y1 = y

reduced_data = training.groupby(training['prognosis']).max()

# Mapping strings to numbers
le = preprocessing.LabelEncoder()
le.fit(y)
y = le.transform(y)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)
testx = testing[cols]
testy = testing['prognosis']
testy = le.transform(testy)

clf1 = DecisionTreeClassifier()
clf = clf1.fit(x_train, y_train)
scores = cross_val_score(clf, x_test, y_test, cv=3)
print(f"Cross-validation score for Decision Tree: {scores.mean():.2f}")

model = SVC()
model.fit(x_train, y_train)
print("SVM model accuracy: ")
print(model.score(x_test, y_test))

importances = clf.feature_importances_
indices = np.argsort(importances)[::-1]
features = cols

def readn(nstr):
    engine = pyttsx3.init()
    engine.setProperty('voice', "english+f5")
    engine.setProperty('rate', 130)
    engine.say(nstr)
    engine.runAndWait()
    engine.stop()

severityDictionary = dict()
description_list = dict()
precautionDictionary = dict()

symptoms_dict = {}

for index, symptom in enumerate(x):
    symptoms_dict[symptom] = index

def calc_condition(exp, days):
    sum = 0
    for item in exp:
        sum = sum + severityDictionary[item]
    if ((sum * days) / (len(exp) + 1)) > 13:
        print("Based on the symptoms and duration, you should seek medical consultation.")
    else:
        print("Your condition may not be severe, but it's advisable to take precautions.")

def getDescription():
    global description_list
    with open('/content/healthcare-chatbot/MasterData/symptom_Description.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        for row in csv_reader:
            _description = {row[0]: row[1]}
            description_list.update(_description)

def getSeverityDict():
    global severityDictionary
    with open('/content/healthcare-chatbot/MasterData/Symptom_severity.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        try:
            for row in csv_reader:
                _diction = {row[0]: int(row[1])}
                severityDictionary.update(_diction)
        except:
            pass

def getprecautionDict():
    global precautionDictionary
    with open('/content/healthcare-chatbot/MasterData/symptom_precaution.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        for row in csv_reader:
            _prec = {row[0]: [row[1], row[2], row[3], row[4]]}
            precautionDictionary.update(_prec)

def getInfo():
    print("------- Welcome to the Healthcare ChatBot -------")
    print("Please enter your name: ", end="")
    name = input("")
    print(f"Hello, {name}. Let's get started.")

def check_pattern(dis_list, inp):
    pred_list = []
    inp = inp.replace(' ', '_')
    patt = f"{inp}"
    regexp = re.compile(patt)
    pred_list = [item for item in dis_list if regexp.search(item)]
    if len(pred_list) > 0:
        return 1, pred_list
    else:
        return 0, []

def sec_predict(symptoms_exp):
    df = pd.read_csv('/content/healthcare-chatbot/Data/Training.csv')
    X = df.iloc[:, :-1]
    y = df['prognosis']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=20)
    rf_clf = DecisionTreeClassifier()
    rf_clf.fit(X_train, y_train)

    symptoms_dict = {symptom: index for index, symptom in enumerate(X)}
    input_vector = np.zeros(len(symptoms_dict))
    for item in symptoms_exp:
        input_vector[[symptoms_dict[item]]] = 1

    return rf_clf.predict([input_vector])

def print_disease(node):
    node = node[0]
    val = node.nonzero()
    disease = le.inverse_transform(val[0])
    return list(map(lambda x: x.strip(), list(disease)))

def tree_to_code(tree, feature_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]

    chk_dis = ",".join(feature_names).split(",")
    symptoms_present = []

    while True:
        print("\nEnter the symptom you are experiencing: ", end="")
        disease_input = input("")
        conf, cnf_dis = check_pattern(chk_dis, disease_input)
        if conf == 1:
            print("Related searches for your input:")
            for num, it in enumerate(cnf_dis):
                print(f"{num}) {it}")
            if num != 0:
                print(f"Select the one you meant (0 - {num}): ", end="")
                conf_inp = int(input(""))
            else:
                conf_inp = 0

            disease_input = cnf_dis[conf_inp]
            break
        else:
            print("Please enter a valid symptom.")

    while True:
        try:
            print("For how many days have you been experiencing these symptoms? ", end="")
            num_days = int(input(""))
            break
        except ValueError:
            print("Invalid input. Please enter a number.")

    def recurse(node, depth):
        indent = "  " * depth
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]

            if name == disease_input:
                val = 1
            else:
                val = 0
            if val <= threshold:
                recurse(tree_.children_left[node], depth + 1)
            else:
                symptoms_present.append(name)
                recurse(tree_.children_right[node], depth + 1)
        else:
            present_disease = print_disease(tree_.value[node])
            red_cols = reduced_data.columns
            symptoms_given = red_cols[reduced_data.loc[present_disease].values[0].nonzero()]
            print("Are you experiencing any of the following symptoms?")
            symptoms_exp = []
            for syms in list(symptoms_given):
                inp = ""
                print(f"{syms}? (yes/no/maybe) ", end='')
                while True:
                    inp = input("").lower()
                    if inp in ["yes", "no", "maybe"]:
                        break
                    else:
                        print("Please provide a valid answer (yes/no/maybe): ", end="")
                if inp == "yes":
                    symptoms_exp.append(syms)
                elif inp == "maybe":
                    symptoms_exp.append(syms)
                    print(f"You mentioned '{syms}' as a maybe. Please confirm if you experience this symptom during further consultation.")

            second_prediction = sec_predict(symptoms_exp)
            calc_condition(symptoms_exp, num_days)
            if present_disease[0] == second_prediction[0]:
                print(f"\nBased on your symptoms, you may have {present_disease[0]}.")
                print(f"Description: {description_list[present_disease[0]]}")
            else:
                print(f"\nBased on your symptoms, you may have {present_disease[0]} or {second_prediction[0]}.")
                print(f"Description for {present_disease[0]}: {description_list[present_disease[0]]}")
                print(f"Description for {second_prediction[0]}: {description_list[second_prediction[0]]}")

            precution_list = precautionDictionary[present_disease[0]]
            print("\nRecommended precautions:")
            for i, j in enumerate(precution_list):
                print(f"{i + 1}. {j}")

            further_action = input("\nWould you like to explore any other options? (yes/no) ").lower()
            if further_action == "yes":
                print("\nOptions:")
                print("1. Speak with a medical professional")
                print("2. Find nearby healthcare facilities")
                print("3. Learn more about the diagnosed condition")
                print("4. Exit")
                choice = input("Enter your choice (1-4): ")
                if choice == "1":
                    print("Connecting you with a medical professional. Please wait...")
                elif choice == "2":
                    print("Finding nearby healthcare facilities. Please enter your location...")
                elif choice == "3":
                    print(f"Here's more information about {present_disease[0]}:")
                    # Add code to display additional information about the diagnosed condition
                elif choice == "4":
                    print("Thank you for using the Healthcare ChatBot. Goodbye!")
                else:
                    print("Invalid choice. Please try again.")

    recurse(0, 1)

getSeverityDict()
getDescription()
getprecautionDict()
getInfo()
tree_to_code(clf, cols)
print("-" * 50)

Cross-validation score for Decision Tree: 0.98
SVM model accuracy: 
1.0
------- Welcome to the Healthcare ChatBot -------
Please enter your name: omar
Hello, omar. Let's get started.

Enter the symptom you are experiencing: fever
Related searches for your input:
0) high_fever
1) mild_fever
Select the one you meant (0 - 1): 0
For how many days have you been experiencing these symptoms? 3
Are you experiencing any of the following symptoms?
fatigue? (yes/no/maybe) yes
cramps? (yes/no/maybe) no
bruising? (yes/no/maybe) yes
obesity? (yes/no/maybe) no
swollen_legs? (yes/no/maybe) no
swollen_blood_vessels? (yes/no/maybe) maybe
You mentioned 'swollen_blood_vessels' as a maybe. Please confirm if you experience this symptom during further consultation.
prominent_veins_on_calf? (yes/no/maybe) yes


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


Your condition may not be severe, but it's advisable to take precautions.

Based on your symptoms, you may have Varicose veins.
Description: A vein that has enlarged and twisted, often appearing as a bulging, blue blood vessel that is clearly visible through the skin. Varicose veins are most common in older adults, particularly women, and occur especially on the legs.

Recommended precautions:
1. lie down flat and raise the leg high
2. use oinments
3. use vein compression
4. dont stand still for long

Would you like to explore any other options? (yes/no) no
--------------------------------------------------
